In [28]:
pwd

'/Users/roeepartoush/Documents/Research/Code/LEanalyze'

In [27]:
cd ../

/Users/roeepartoush/Documents/Research/Code/LEanalyze


In [29]:
# == IMPORTS ==

# basic
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
from copy import deepcopy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# astropy
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle, Latitude, Longitude
import astropy.units as u
from astropy.io import ascii
from astropy.time import Time
from astropy import wcs
from astropy.visualization import ZScaleInterval

# Modeling
from scipy.ndimage import gaussian_filter
from scipy import signal
from scipy import ndimage
from scipy.optimize import curve_fit
from lmfit import Model
import lmfit
# import corner # currently unused

# local modules
import File2Data as F2D
import DustFit as DFit
import LEtoolbox as LEtb
import LEplots
import run_global_vars

In [15]:
# Region file format: DS9 version 4.1

slits = []
PAs = []
LNs = []
WDs = []
dates = []

# text(0:49:25.1165,+58:45:33.501) text={Sep12}
# box(0:49:27.6670,+58:45:31.060,1.000",120.000",79)
slits.append(SkyCoord("0h49m27.6670s","+58d45m31.060s",frame='fk5'))
PAs.append(Angle(79,u.deg))
LNs.append(Angle(120,u.arcsec))
WDs.append(Angle(1,u.arcsec))
dates.append('2012-09-18T00:00:00')

# text(0:49:31.1429,+58:45:45.566) textangle=3.9809219e-05 text={Oct 13}
# box(0:49:33.4240,+58:46:19.210,1.000",100.000",352)
slits.append(SkyCoord("0h49m33.4240s","+58d46m19.210s",frame='fk5'))
PAs.append(Angle(352,u.deg))
LNs.append(Angle(100,u.arcsec))
WDs.append(Angle(1,u.arcsec))
dates.append('2013-10-05T00:00:00')

# text(0:49:32.1203,+58:46:43.698) text={Dec 13}
# box(0:49:29.6560,+58:45:04.440,1.000",200.000",24.5)
slits.append(SkyCoord("0h49m29.6560s","+58d45m04.440s",frame='fk5'))
PAs.append(Angle(24.5,u.deg))
LNs.append(Angle(200,u.arcsec))
WDs.append(Angle(1,u.arcsec))
dates.append('2013-12-28T00:00:00')



In [16]:
Orig = SkyCoord("0h49m27.6670s","+58d45m31.060s",frame='fk5')
print(Orig.ra.deg)
print(Orig.dec.deg)

12.365279166666665
58.758627777777775


In [17]:
Orgs=SkyCoord(slits,frame='fk5',unit=(u.deg, u.deg))
PA = (Angle(PAs)+Angle(180,u.deg)).wrap_at(360 * u.deg)
Ln = Angle(LNs,u.arcsec)
Wd = Angle(WDs,u.arcsec)
Dt = Time(dates, format='isot', scale='utc')


clmns = ['Orig', 'PA', 'Length', 'Width']
slitFPdf = pd.DataFrame(index=np.arange(len(Orgs)), columns=clmns, data = [(Orgs[i],PA[i],Ln[i],Wd[i]) for i in np.arange(len(Orgs))])

spectra_df = slitFPdf.copy()
spectra_df.insert(spectra_df.shape[1],'date',Dt)

In [19]:
def df2writable(spectra_df):
    spectra_df_wr = spectra_df.copy()
    spectra_df_wr.insert(1,'RA',None)
    spectra_df_wr.insert(2,'DEC',None)
    for i in np.arange(len(spectra_df_wr)):
        Orig = spectra_df_wr.at[i,'Orig']
        spectra_df_wr.at[i,'RA'] = Orig.ra.to_string()
        spectra_df_wr.at[i,'DEC'] = Orig.dec.to_string()
    spectra_df_wr.drop(columns='Orig',inplace=True)
    return spectra_df_wr

def writable2df(spectra_df_wr):
    spectra_df = spectra_df_wr.copy()
    spectra_df.insert(0,'Orig',None)
    for i in np.arange(len(spectra_df)):
        ra = spectra_df.at[i,'RA']
        dec = spectra_df_wr.at[i,'DEC']
        Orig = SkyCoord(ra,dec,frame='fk5')
        spectra_df.at[i,'Orig'] = Orig

        PA = spectra_df.at[i,'PA']
        spectra_df.at[i,'PA'] = Angle(PA)
        
        Ln = spectra_df.at[i,'Length']
        spectra_df.at[i,'Length'] = Angle(Ln)
        
        Wd = spectra_df.at[i,'Width']
        spectra_df.at[i,'Width'] = Angle(Wd)
        
        Dt = spectra_df.at[i,'date']
        spectra_df.at[i,'Width'] = Time(Dt, format='isot', scale='utc')
    
    spectra_df.drop(columns=['RA','DEC'],inplace=True)
    return spectra_df

In [20]:
# spectra_df_wr = convert2writable(
spectra_df_wr = spectra_df.copy()
Orig = spectra_df_wr.at[0,'Orig']
Orig.ra.to_string()
# Orig.dec.to_string()

'12d21m55.005s'

In [21]:
df2writable(spectra_df).to_csv('spectra.csv',index=False)

In [22]:
Angle('240d00m00s').deg

240.0

In [23]:
df = pd.read_csv('spectra.csv')

In [24]:
type(writable2df(df).loc[0,'PA'])

astropy.coordinates.angles.Angle